In [5]:
from keras.models import Sequential
from keras.datasets import mnist
from keras import layers
import numpy as np

# Fetching the Dataset and preprocessing:

In [6]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

#reshaping the data to n*784 (from n*28*28) so that we can apply  
#feature scaling feature scaling only works for (at max) 2d data
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2])
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2])

#feature scaling:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

#reshaping x to n*28*28*1
x_train=x_train.reshape(x_train.shape[0],int(x_train.shape[1]**0.5),int(x_train.shape[1]**0.5),1)   #1 channel only
x_test=x_test.reshape(x_test.shape[0],int(x_test.shape[1]**0.5),int(x_test.shape[1]**0.5),1)        #1 channel only

#one hot encoding y_train and y_test:
from keras.utils.np_utils import to_categorical   

y_train=to_categorical(y_train,num_classes=len(set(y_train)))
y_test=to_categorical(y_test,num_classes=len(set(y_test)))

In [7]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000, 10)
(10000, 10)


In [ ]:
fig=plt.figure(figsize=(8,8))                      #plotting the images an 8*8 grid
for i in range(64):                                #we are only plotting 64 images instead of entire 50000
    ax=fig.add_subplot(8,8,i+1)
    ax.imshow(x_train[i])                                    
plt.show()

# Creating the Model:

In [4]:
model=Sequential()

# Constants:

In [5]:
#intializing all the constant parameters
input_width=28
input_height=28
input_channels=1                            #input image is 28*28*1
input_pixels=784

n_conv1=32                                  #no. of units in conv layer 1
n_conv2=64

conv1_k=5                                   #filter size for conv layer 1
conv2_k=5

conv1_stride=1                              #stride for conv layer 1
conv2_stride=1

pool1_k=2                                   #pool size for pooling layer 1
pool2_k=2

n_hidden=1024                               #no. of units in hidden layer
n_output=10                                 #no. of units in output layer(there are 10 classes)

input_size_to_hiddenlayer=(input_height//(pool1_k*pool2_k))*(input_width//(pool1_k*pool2_k))*n_conv2

# Defining the Architecture:

In [6]:
#convolution layer 1:
conv1=layers.Conv2D(filters=n_conv1,padding="same",
                    strides=[conv1_stride,conv1_stride],
                    activation="relu",use_bias=True,
                    kernel_size=[conv1_k,conv1_k],
                    input_shape=(input_height,input_width,input_channels))
#filters represent the no. of units in a layer, strides: strides along heights and width,kernel_size:size of filter(K*K)
#input_shape represents te shape of 1 input image(1 data point), it is only required for the first layer

#convolution layer 2:
conv2=layers.Conv2D(filters=n_conv2,padding="same",
                    strides=[conv2_stride,conv2_stride],
                    activation="relu",use_bias=True,
                    kernel_size=[conv2_k,conv2_k],
                    input_shape=(input_height,input_width,input_channels))

#pooling layer 1:
pool1=layers.MaxPooling2D(pool_size=[pool1_k,pool1_k],strides=[pool1_k,pool1_k],padding="same")

#pooling layer 2:
pool2=layers.MaxPooling2D(pool_size=[pool2_k,pool2_k],strides=[pool2_k,pool2_k],padding="same")

#flatten layer:                        #to flatten the output images of pool2 into 1d array
flatten=layers.Flatten()

#Hidden layer:
hidden=layers.Dense(units=n_hidden,use_bias=True,activation="relu")

#Dropout layer:
dropout=layers.Dropout(rate=0.2)

#Output layer:
output=layers.Dense(units=n_output,use_bias=True,activation="softmax")

model.add(conv1)
model.add(pool1)
model.add(conv2)
model.add(pool2)
model.add(flatten)
model.add(hidden)
model.add(dropout)
model.add(output)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model.layers                          
#if u run above cell multiple times, same layers 
#will get added again, so if u have made changes 
#to above cell,then run again from model=Sequential()

# Compiling the Model:

In [8]:
model.compile(optimizer="adadelta",loss="categorical_crossentropy",metrics=["accuracy"])

# Fit:

In [9]:
model.fit(x_train,y_train,epochs=12,batch_size=128,validation_data=(x_test,y_test))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
 1408/60000 [..............................] - ETA: 3:58 - loss: 1.5736 - acc: 0.4844

KeyboardInterrupt: 

In [191]:
pred=model.predict(x_test)
pred                                      
#see that all values except 1 in each row are very less(0.00000...)
#and 1 value(our predicted class) is almost (or equal to) one.

array([[7.2779506e-20, 6.1409880e-18, 6.8149239e-19, ..., 1.0000000e+00,
        1.7213005e-21, 6.0754522e-15],
       [3.2433104e-21, 1.1200953e-17, 1.0000000e+00, ..., 5.4996862e-26,
        1.5165027e-26, 1.4131293e-29],
       [8.7301531e-14, 1.0000000e+00, 4.9457324e-13, ..., 1.0654031e-10,
        1.6450252e-10, 1.5151919e-16],
       ...,
       [2.0988546e-27, 2.4161638e-19, 4.7742335e-28, ..., 1.3944051e-19,
        2.6215620e-18, 1.1066008e-20],
       [8.4826073e-18, 8.4630632e-23, 1.9569004e-26, ..., 1.5343478e-20,
        1.9292499e-07, 1.8196381e-20],
       [3.9612351e-16, 1.6849093e-18, 1.8468819e-15, ..., 1.2090598e-21,
        4.6556882e-15, 9.4955225e-22]], dtype=float32)

In [192]:
score=model.evaluate(x_test,y_test)
score                         #will give u the loss, and the metrics u asked for in compile()

10000/10000 [==============================] - 7s 652us/step


[0.03618634108330202, 0.9928]